# PythonでAPIを叩く

## Parse .bibtex

中身はシンプルに
```
@inproceedings{vaswani2017attention,
  title={Attention is all you need},
  author={Vaswani, Ashish and Shazeer, Noam and Parmar, Niki and Uszkoreit, Jakob and Jones, Llion and Gomez, Aidan N and Kaiser, {\L}ukasz and Polosukhin, Illia},
  booktitle={Advances in neural information processing systems},
  pages={5998--6008},
  year={2017}
}
```
みたいなbibがいくつか入ってる．

In [1]:
import re 
from collections import defaultdict

# bibtexをdictのlistにparseする
def parse_bibtex(fname):
    # Open bibtex file
    with open(fname) as f:
        lines = ''.join(f.readlines()).rstrip()
        
    # split bibtex into chunks of individual refs
    chunks = list(filter(None, lines.split('@')))
    
    # list of dicts to store the parsed refs
    bibs = [defaultdict(lambda : '') for _ in chunks]

    # Loop over each chunk
    for idx, chunk in enumerate(chunks):
        
        # Get the first ref 
        match = re.search(r'{(.*?),', chunk)
        if match:
            bib_ref = match.group(1)
            bibs[idx]['ref'] = bib_ref
            bibs[idx]['type'] = chunk[:chunk.index('{')]
            
            i = chunk.index('{') + len(bib_ref)
            while i <= len(chunk):
                # Capture LHS of =
                m = re.search(r'(\w*?)=', chunk[i:])
                
                # Capture RHS of =
                if m:
                    # Make a stack to parse parenthesis
                    stack = []
                    
                    # Push the first element manually.
                    i += chunk[i:].index('{')+1
                    stack.append(0)
                    
                    # Loop and parse
                    while len(stack) > 0:
                        c = chunk[i]
                        if c == '{':
                            stack.append(0)
                        elif c == '}':
                            stack.pop()
                        else:
                            bibs[idx][m.group(1)] += c
                        i += 1
                i += 1
    return bibs
            
bibs = parse_bibtex('sample.bib')

for bib in bibs:
    for key, value in bib.items():
        print(key, ':', value)
    print()

ref : vaswani2017attention
type : inproceedings
title : Attention is all you need
author : Vaswani, Ashish and Shazeer, Noam and Parmar, Niki and Uszkoreit, Jakob and Jones, Llion and Gomez, Aidan N and Kaiser, \Lukasz and Polosukhin, Illia
booktitle : Advances in neural information processing systems
pages : 5998--6008
year : 2017

ref : lecun2015deep
type : article
title : Deep learning
author : LeCun, Yann and Bengio, Yoshua and Hinton, Geoffrey
journal : nature
volume : 521
number : 7553
pages : 436--444
year : 2015
publisher : Nature Publishing Group

ref : gers1999learning
type : article
title : Learning to forget: Continual prediction with LSTM
author : Gers, Felix A and Schmidhuber, J\"urgen and Cummins, Fred
year : 1999
publisher : IET



## bibtexを元に検索する

以下の3つの内どれかが必要

- Arxiv ID
- DOI
- PMC
- Pubmed

### Arxiv ID の取得 (Arxiv API)

In [2]:
import arxiv, os

q = 'ti:{0}'
for bib in bibs:
    query =  q.format(
        bib.get('title'),
    )
    res = arxiv.query(
        query=query,
        max_results=3,
        sort_by="relevance")
    print('Search results for: "{}"'.format(bib.get('title')))
    for r in res:
        print('\t', r.get('title')[:20], r.get('id'), os.path.basename(r.get('id')))

Search results for: "Attention is all you need"
	 Attention Is All You http://arxiv.org/abs/1706.03762v5 1706.03762v5
	 Attention is all you http://arxiv.org/abs/1906.02792v1 1906.02792v1
	 You May Not Need Att http://arxiv.org/abs/1810.13409v1 1810.13409v1
Search results for: "Deep learning"
	 Deep Prior http://arxiv.org/abs/1712.05016v2 1712.05016v2
	 Deep Learning http://arxiv.org/abs/1807.07987v2 1807.07987v2
	 Deep Graphs http://arxiv.org/abs/1806.01235v1 1806.01235v1
Search results for: "Learning to forget: Continual prediction with LSTM"
	 Learning To Score Ol http://arxiv.org/abs/1611.05125v3 1611.05125v3
	 Learning to Skim Tex http://arxiv.org/abs/1704.06877v2 1704.06877v2
	 Learning Natural Lan http://arxiv.org/abs/1512.08849v2 1512.08849v2


### DOIの取得 (Science Direct API)

In [3]:
from elsapy.elsclient import ElsClient
from elsapy.elssearch import ElsSearch
import json
    
## Load configuration
with open("config.json") as conf:
    config = json.load(conf)

## Initialize client
client = ElsClient(config['apikey'])

In [4]:
## Initialize doc search object using ScienceDirect and execute search, 
#   retrieving all results

for bib in bibs:
    query =  q.format(bib.get('title'))
    print('Search results for: "{}"'.format(query))
    try:
        doc_srch = ElsSearch(query,'sciencedirect')
        doc_srch.execute(client, get_all = False)
        res = doc_srch.results_df

        res= doc_srch.results_df
        for _, r in res.iterrows():
            print('\t', r.get('dc:title')[:30], r.get('dc:identifier'))
            print('\t\t', r.get('prism:url'))
            if _ > 3: break
    except:
        print('\tOops, Some sort of error')

Search results for: "ti:Attention is all you need"
	Oops, Some sort of error
Search results for: "ti:Deep learning"
	Oops, Some sort of error
Search results for: "ti:Learning to forget: Continual prediction with LSTM"
	Oops, Some sort of error


### DOIの取得 (cross ref API)

In [5]:
import pandas as pd
from crossref.restful import Works

works = Works()

for bib in bibs:
    query =  q.format(bib.get('title'))
    print('Search results for: "{}"'.format(query))
    res = iter(works.query(query).sort())
    try:
        for _, r in enumerate(res):
            print('\t', r.get('title')[0][:20], r.get('DOI'))
            print('\t\t', r.get('URL'))
            if _ > 3: break
    except:
        print('\tOops, Some sort of error')

Search results for: "ti:Attention is all you need"
	 Attention All Survey 10.1108/02630809410074466
		 http://dx.doi.org/10.1108/02630809410074466
	 Attention Is (not) A 10.18653/v1/p19-1477
		 http://dx.doi.org/10.18653/v1/p19-1477
	 Knowledge Is All You 10.1111/phis.12060
		 http://dx.doi.org/10.1111/phis.12060
	 All You Need Is Love 10.1093/oso/9780190460723.003.0003
		 http://dx.doi.org/10.1093/oso/9780190460723.003.0003
	 All you need for all 10.1038/bdjteam.2016.20
		 http://dx.doi.org/10.1038/bdjteam.2016.20
Search results for: "ti:Deep learning"
	 Deep drawability of  10.1063/1.5008085
		 http://dx.doi.org/10.1063/1.5008085
	 Computer Vision, Dee 10.14293/s2199-1006.1.sor-uncat.clzwyuz.v1
		 http://dx.doi.org/10.14293/s2199-1006.1.sor-uncat.clzwyuz.v1
	 Introduction to Deep 10.7551/mitpress/11171.003.0004
		 http://dx.doi.org/10.7551/mitpress/11171.003.0004
	 A Brief History of D 10.7551/mitpress/11171.003.0007
		 http://dx.doi.org/10.7551/mitpress/11171.003.0007
	 The Future o

## ReferencesTo and ReferencesFrom の取得

ProphyAPIに関するドキュメントはないし，継続して使える保証がないのは心配．

GraphQLを使ってるみたいで，https://www.prophy.science/api/graphql/ からちょっとしたドキュメントが見れる．

Auth関係がないみたいだったから，Prophy管理者に使用許可→取得済み

In [6]:
import requests, json

r = requests.get("https://www.prophy.science/api")
print(r) # Response 200 = OK
print(r.content.decode("utf-8"))

<Response [200]>
{
  "msg": "Welcome to Prophy API", 
  "time": "Wed, 22 Jan 2020 16:33:10 GMT"
}



In [7]:
url = 'https://www.prophy.science/api/graphql'
headers= {'Accept': 'application/vnd.cap-collectif.preview+json'}
queries = {'query' :'query {article(id:430) {id}}'}

r = requests.post(url, headers=headers, json=queries)
id = json.loads(r.content).get('data').get('article').get('id')
print(id)

430


In [8]:
url = 'https://www.prophy.science/api/articles/{0}/full'.format(id)
queries = {'query' :'query {article(id:430) {id}}'}

r = requests.get(url, headers=headers, json=queries)
res = json.loads(r.content)
referencesFrom = res['referencesFrom']
referencesTo = res['referencesTo']

In [9]:
for item in referencesTo['references']:
    print(item['title'][:30], '...')

Towards a Better Understanding ...
Neonatal Resveratrol and Nicot ...
Perilipin-2 deletion promotes  ...
Genetics of metabolic syndrome ...
Adipocyte Liver Kinase b1 Supp ...
Human brown adipose tissue as  ...
Adipose tissue macrophages dev ...
Targeting PPAR γ  in the epige ...
Global IP6K1 deletion enhances ...
PDGFRα controls the balance of ...


In [10]:
for item in referencesFrom['references']:
    print(item['title'][:30], '...')

Ectopic brown adipose tissue i ...
Inactivation of the Mitochondr ...
Allelic forms of a gene contro ...
Innervation of mammalian white ...
The Uncoupling Protein 1 Gene  ...
The landscape of genetic compl ...
Brown adipose tissue hyperplas ...
Brown adipose tissue: function ...
Tissue-specific and beta-adren ...
FOXC2 is a winged helix gene t ...
Beta 3-adrenergic receptor sti ...
RIP140-Targeted Repression of  ...
Adipocyte differentiation and  ...
Strain-Specific Response to  3 ...
Dietary fat interacts with QTL ...
Identification and Importance  ...
Mice lacking mitochondrial unc ...
Genetic regulation of traits e ...
Ontogeny and perinatal modulat ...
Emergence of brown adipocytes  ...
Target of rapamycin-mediated a ...
Regulatory circuits controllin ...
Effect of CL-316,243, a thermo ...
Multilocular fat cells in WAT  ...
In developing brown adipose ti ...
The LIS1-related protein NUDF  ...
Uncoupling protein in embryoni ...
Transcriptional Control of Bro ...
Regulation of the br